In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("preprocessing_1.csv", index_col=0)
df.head(10)

,Make,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City,Fuel Consumption Hwy,Fuel Consumption Comb,CO2 Emissions(g/km),MakeAvg,Vehicle ClassAvg,TransmissionAvg,Fuel TypeAvg,Engine/Cylinders
6,GONDA,FULL-SIZE,1.8,4.0,AV7,X,13.297872,41.694209,7.843137,174.0,193.130130,293.769947,232.318306,267.157393,0.450000
7,LAND CRAWLER,SUV - SMALL,2.0,4.0,AS8,Z,67.153800,48.543689,18.330000,246.0,377.249357,197.291539,247.833952,276.532490,0.500000
11,BMV,SUBCOMPACT,3.0,6.0,M6,Z,5.260389,93.416496,6.990000,236.0,208.438650,211.152576,181.703590,276.532490,0.500000
12,TOYOTI,MINIVAN,2.0,4.0,AS6,X,45.454545,18.867925,27.940000,260.0,282.353837,291.312755,196.684061,267.157393,0.500000
17,FOLD,SUV - STANDARD,2.3,4.0,AS6,X,19.920319,14.245014,16.891892,263.0,193.474820,232.319654,196.684061,267.157393,0.575000
19,CHEVO,SUV - STANDARD,5.3,8.0,A6,X,6.609385,2.376991,20.299029,335.0,225.053859,232.319654,300.656411,267.157393,0.662500
21,CHEVO,COMPACT,1.4,4.0,AS6,X,9.210000,88.218844,10.000000,130.0,225.053859,253.192320,196.684061,267.157393,0.350000
22,GONDA,STATION WAGON - SMALL,3.5,6.0,M6,X,8.610000,11.580000,9.737098,158.0,193.130130,246.618504,181.703590,267.157393,0.583333
29,LECUS,SUV - SMALL,3.5,6.0,AS8,X,5.624297,10.720000,7.142857,259.0,251.244891,197.291539,247.833952,267.157393,0.583333
36,BMV,COMPACT,3.0,6.0,A8,Z,64.935065,16.112208,33.941483,188.0,208.438650,253.192320,295.519132,276.532490,0.500000


In [9]:
df.columns

Index(['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission',
       'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy',
       'Fuel Consumption Comb', 'CO2 Emissions(g/km)', 'MakeAvg',
       'Vehicle ClassAvg', 'TransmissionAvg', 'Fuel TypeAvg',
       'Engine/Cylinders'],
      dtype='object')

In [10]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(4, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])
# col_train=['Make', 'Vehicle Class', 'Engine Size(L)', 'Cylinders', 'Transmission',
#        'Fuel Type', 'Fuel Consumption City', 'Fuel Consumption Hwy',
#        'Fuel Consumption Comb', 'CO2 Emissions(g/km)', 'MakeAvg',
#        'Vehicle ClassAvg', 'TransmissionAvg', 'Fuel TypeAvg']
col_train=['Cylinders', 'Fuel Consumption City', 'Fuel Consumption Hwy',
       'Fuel Consumption Comb', 'MakeAvg',
       'Vehicle ClassAvg', 'TransmissionAvg', 'Fuel TypeAvg']
# model.compile(loss="mae", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=tf.keras.metrics.RootMeanSquaredError())
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.RootMeanSquaredError())

history=model.fit(df[col_train], df["CO2 Emissions(g/km)"], epochs=50, validation_split=0.2, shuffle=False)

Epoch 1/50
1162/1162 [==============================] - 10s 7ms/step - loss: 62368.0859 - root_mean_squared_error: 249.7360 - val_loss: 59652.9062 - val_root_mean_squared_error: 244.2394
Epoch 2/50
1162/1162 [==============================] - 7s 6ms/step - loss: 56204.7227 - root_mean_squared_error: 237.0753 - val_loss: 51492.2773 - val_root_mean_squared_error: 226.9191
Epoch 3/50
1162/1162 [==============================] - 7s 6ms/step - loss: 46787.7305 - root_mean_squared_error: 216.3047 - val_loss: 41214.8164 - val_root_mean_squared_error: 203.0143
Epoch 4/50
1162/1162 [==============================] - 7s 6ms/step - loss: 36150.5977 - root_mean_squared_error: 190.1331 - val_loss: 30528.5195 - val_root_mean_squared_error: 174.7241
Epoch 5/50
1162/1162 [==============================] - 8s 7ms/step - loss: 25758.9746 - root_mean_squared_error: 160.4960 - val_loss: 20706.1094 - val_root_mean_squared_error: 143.8962
Epoch 6/50
1162/1162 [==============================] - 8s 7ms/step -

KeyboardInterrupt: 

In [ ]:
loss = np.sqrt(history.history['loss'])
val_loss = np.sqrt(history.history['val_loss'])

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

KeyError: 'val_loss'